In [1]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from pprint import pprint
from datetime import datetime


sys.path.append(str(Path("..").resolve()))
from src import *

# Delete Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [ ]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/5_optimized/omero_museum"

In [2]:
connector = MongoDBConnector("omero_museum", silent=True)
db = connector.db

### DL1: _Delete a posted comment_

In [6]:
artwork = db.artworks.find_one({"comments_star_3.0": {"$exists": True}})
artwork_id = artwork["_id"] # type: ignore
comment_id = artwork["comments_star_3"][0]["_id"] # type: ignore

db.artworks.update_many(
    {},
    {
        "$pull": {
            "comments_star_1": {"_id": comment_id},
            "comments_star_2": {"_id": comment_id},
            "comments_star_3": {"_id": comment_id},
            "comments_star_4": {"_id": comment_id},
            "comments_star_5": {"_id": comment_id},
        }
    },
)

UpdateResult({'n': 21, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### DL2: _Delete an artwork_

In [ ]:
db.artworks.delete_one({"_id": "La Notte Stellata"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

### DL4: _Delete a shift of an employee_

In [7]:
employee_id = db.roles.find_one()["_id"]  # type: ignore
day_of_week = "martedì"

db.roles.update_one(
    {"_id": employee_id},
    {"$pull": {"shifts": {"day_of_week": day_of_week}}},
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### DL6: _Delete a message_

In [8]:
customer = db.visitors.find_one({"chat.old_messages.0": {"$exists": True}})
customer_id = customer["_id"]  # type: ignore
message_id = customer["chat"]["old_messages"][0]  # type: ignore

db.visitors.update_many(
    {"_id": customer_id},
    {"$pull": {"chat.old_messages": {"_id": message_id}}},
)
db.messages.delete_one({"_id": message_id})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)